# Non-working tools

## Snakefood
https://github.com/blais/snakefood

Did not work because:

"Python-3.x is NOT supported, but there have been a few efforts to port to it. I never had time to fully port it; it would require a fair bit of a rewrite to do so IMO."

## PyCallGraph

Couldn't get it to work. Chased down a never-ending rabbithole.

1. Can't find entrypoint / main. Most likely due to the complexity of how the project is run.
2. If we try to help it and give some modules a main(), it then can't figure out the dependencies
3. If we add the dependencies, they haven't been setup properly.
4. Setting them up properly is too hard.

# Initial thoughts

We want to start with some raw metrics like count of files, lines of code.
This is important to give us some context.
Example, if a package consists of 1 file, with 5 lines of code. Then naturally that package will have lower metrics later on, such as cyclomatic dependencies, etc.


Some tools need a __init__ So make a empty .py file in API and Frontend.

# Dependencies setup

In [16]:
import os
import sys
import subprocess

def enforce_numpy_version():
    """
    Ensure that NumPy is installed and is a version below 2.x.
    If a 2.x version (or higher) is found, downgrade to a 1.x version
    and instruct the user to restart the kernel manually if running interactively.
    """
    try:
        import numpy as np
        major_version = int(np.__version__.split('.')[0])
        if major_version >= 2:
            print(f"Detected NumPy version {np.__version__}. Downgrading to numpy<2...")
            subprocess.check_call([sys.executable, "-m", "pip", "install", "--upgrade", "numpy<2"])
            if "ipykernel" in sys.modules:
                print("Please restart your Jupyter kernel manually after downgrading NumPy.")
                sys.exit(0)
            else:
                print("Restarting the script after downgrading NumPy...")
                os.execv(sys.executable, [sys.executable] + sys.argv)
        else:
            print(f"NumPy version {np.__version__} is compatible (version <2).")
    except ImportError:
        print("NumPy is not installed. Installing numpy<2...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", "numpy<2"])
        if "ipykernel" in sys.modules:
            print("Please restart your Jupyter kernel manually after installing NumPy.")
            sys.exit(0)
        else:
            print("Restarting the script after installing NumPy...")
            os.execv(sys.executable, [sys.executable] + sys.argv)

def install_package(package):
    """
    Try to import a package. If it’s not found, install it using pip.
    If pip fails for any reason, catch the exception and print an error message.
    """
    try:
        # Replace dashes with underscores in case the module name differs from the package name.
        __import__(package.replace('-', '_'))
        print(f"{package} is already installed.")
    except ImportError:
        print(f"Installing {package}...")
        try:
            subprocess.check_call([sys.executable, "-m", "pip", "install", package])
        except subprocess.CalledProcessError as e:
            print(f"Failed to install {package}. Error: {e}")

if __name__ == "__main__":
    # Ensure NumPy is compatible before proceeding.
    enforce_numpy_version()
    
    # List of packages to install.
    packages = [
        'radon',
        'matplotlib',
        'pandas',
        'networkx',
        'pydot',
        'squarify',
        'pylint',
        'graphviz',
        'pyan3',
        'prospector'
    ]
    
    for pkg in packages:
        install_package(pkg)
    
    print("All packages have been checked and installed as needed.")


NumPy version 1.26.4 is compatible (version <2).
radon is already installed.
matplotlib is already installed.
pandas is already installed.
networkx is already installed.
pydot is already installed.
squarify is already installed.
pylint is already installed.
graphviz is already installed.
Installing pyan3...
Failed to install pyan3. Error: Command '['c:\\Python312\\python.exe', '-m', 'pip', 'install', 'pyan3']' returned non-zero exit status 1.
Installing prospector...
Failed to install prospector. Error: Command '['c:\\Python312\\python.exe', '-m', 'pip', 'install', 'prospector']' returned non-zero exit status 1.
All packages have been checked and installed as needed.


In [5]:
import datetime
import os
import sys
import subprocess
import json
import matplotlib.pyplot as plt
from pathlib import Path
from collections import defaultdict
import networkx as nx

# Path setup

In [14]:
TOOLS_DIR = os.getcwd()  # Expected: ...\Tools
BASE_DIR = os.path.abspath(os.path.join(TOOLS_DIR, '..'))
DATA_DIR = os.path.join(BASE_DIR, 'Data')

# Our repos to analyse:
API_DIR = os.path.join(DATA_DIR, 'api')
WEB_DIR = os.path.join(DATA_DIR, 'web')

# Analyzation project(s) setup

In [15]:
def clone_repo(repo_url, target_dir):
    """
    Clone a git repository from repo_url into target_dir.
    Also create an empty __init__.py file in the repository's root.
    """
    # Ensure the parent directory exists.
    parent_dir = os.path.dirname(target_dir)
    if not os.path.exists(parent_dir):
        os.makedirs(parent_dir)
        print(f"Created parent directory: {parent_dir}")
    
    # Clone the repository if not already present.
    if not os.path.exists(target_dir):
        subprocess.run(["git", "clone", repo_url, target_dir], check=True)
        print(f"Repository cloned from {repo_url} into {target_dir}")
    else:
        print(f"Repository already exists at {target_dir}")
    
    # Create an empty __init__.py file in the repo root if it doesn't already exist.
    init_file = os.path.join(target_dir, '__init__.py')
    if not os.path.exists(init_file):
        open(init_file, 'w').close()  # This creates an empty file.
        print(f"Empty __init__.py created in {target_dir}")
    else:
        print(f"__init__.py already exists in {target_dir}")

# Clone the zeeguu/api repository.
clone_repo("https://github.com/zeeguu/api", API_DIR)

# Clone the zeeguu/web repository.
clone_repo("https://github.com/zeeguu/web", WEB_DIR)

Repository already exists at c:\Users\jgrum\Documents\Programming\ArchitecturalRecovery\Data\api
__init__.py already exists in c:\Users\jgrum\Documents\Programming\ArchitecturalRecovery\Data\api
Repository cloned from https://github.com/zeeguu/web into c:\Users\jgrum\Documents\Programming\ArchitecturalRecovery\Data\web
Empty __init__.py created in c:\Users\jgrum\Documents\Programming\ArchitecturalRecovery\Data\web


# Architectural-Lens

# GitTruck

# Prospector

# Pyan

# Pyreverse

# Radon

## Basic project setup

In [9]:
def list_directories(root_dir):
    dirs = []
    for dirpath, dirnames, _ in os.walk(root_dir):
        # Exclude hidden directories
        dirnames[:] = [d for d in dirnames if not d.startswith('.')]
        dirs.append(os.path.relpath(dirpath, root_dir))
    return sorted(dirs)

print("API (Backend) Structure:")
for d in list_directories(os.path.join('..', 'Data', 'api')):
    print(d)

API (Backend) Structure:
